In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df_clientes = pd.read_csv('../../dados/credit_score/df_clientes_variavel_target.csv')

# Train Test Split

In [3]:
SEED = 42

In [4]:
x = df_clientes.drop(columns=['Risco_de_credito'])
y = df_clientes['Risco_de_credito']

In [5]:
df_train, df_test = train_test_split(df_clientes, test_size=0.25, random_state=SEED, stratify=df_clientes['Risco_de_credito'])

In [6]:
print(df_train.shape, df_test.shape)

(21307, 16) (7103, 16)


In [7]:
df_train.to_csv('../../dados/credit_score/df_clientes_train.csv', index=False)
df_test.to_csv('../../dados/credit_score/df_clientes_test.csv', index=False)

# Criação pipeline

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from imblearn.over_sampling import SMOTE

## Drop ID_Cliente

In [9]:
class DropFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, feature_to_drop='ID_Cliente') -> None:
        self.feature_to_drop = feature_to_drop

    def fit(self, df):
        return self
    
    def transform(self, df):
        if self.feature_to_drop in df.columns:
            drop_df = df.drop(columns=[self.feature_to_drop])
            return drop_df
        else:
            print(f'Variável {self.feature_to_drop} não encontrada no dataframe')
            return df

## MinMaxScaler

In [10]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self, min_max_scaler=['Idade', 'Anos_empregado', 'Tamanho_familia','Rendimento_anual']):
        self.min_max_scaler = min_max_scaler

    def fit(self, df):
        return self
    
    def transform(self, df):
        if set(self.min_max_scaler).issubset(df.columns):
            scaler = MinMaxScaler()
            df[self.min_max_scaler] = scaler.fit_transform(df[self.min_max_scaler])

            return df
        else:
            print(f'Colunas {[col for col in self.min_max_scaler if col not in df.columns]} não encontradas')
            return df

## OneHotEncoding

In [11]:
class OneHotEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, one_hot_encoder=['Categoria_de_renda','Estado_civil','Moradia','Ocupacao']):
        self.one_hot_encoder = one_hot_encoder

    def fit(self, df):
        return self
    
    def transform(self, df):
        if set(self.one_hot_encoder).issubset(df.columns):
            def one_hot_enc(df, one_hot_encoder):
                one_hot_enc = OneHotEncoder()
                one_hot_enc.fit(df[one_hot_encoder])
                feature_names = one_hot_enc.get_feature_names_out(one_hot_encoder)
                df = pd.DataFrame(one_hot_enc.transform(df[self.one_hot_encoder]).toarray(),
                                columns=feature_names, index=df.index)
                df[feature_names] = df[feature_names].astype(int)
            
                return df
            
            def concat_result(df, one_hot_enc_df, one_hot_encoder):
                other_features = [column for column in df.columns if column not in one_hot_encoder]
                df_concat = pd.concat([df[other_features], one_hot_enc_df], axis=1)

                return df_concat
            
            df_OneHotEncoding = one_hot_enc(df, self.one_hot_encoder)
            df_final = concat_result(df, df_OneHotEncoding, self.one_hot_encoder)

            return df_final
            
        else:
            print(f'Colunas {[col for col in self.one_hot_encoder if col not in df.columns]} não encontradas')
            return df


## OrdinalEncoding

In [12]:
class OrdinalFeature(BaseEstimator, TransformerMixin):
    def __init__(self, ordinal_feature=['Grau_escolaridade']):
        self.ordinal_feature = ordinal_feature

    def fit(self, df):
        return self
    
    def transform(self, df):
        if self.ordinal_feature[0] in df.columns:
            ordinal_encoder = OrdinalEncoder(dtype=int)
            df[self.ordinal_feature] = ordinal_encoder.fit_transform(df[self.ordinal_feature])

            return df
        else:
            print(f'Variável {self.ordinal_feature} não encontrada no dataframe!')
            return df

## Oversample

In [13]:
class OverSample(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, df):
        return self
    
    def transform(self, df):
        oversampler = SMOTE(sampling_strategy='minority')
        x_bal, y_bal = oversampler.fit_resample(df.drop(columns=['Risco_de_credito']), df['Risco_de_credito'])
        df_bal = pd.concat([pd.DataFrame(x_bal), pd.DataFrame(y_bal)], axis=1)

        return df_bal

# Rodando pipeline

In [14]:
from sklearn.pipeline import Pipeline

In [15]:
def pipeline_ml(df):
    pipeline = Pipeline([
        ('feature_dropper', DropFeatures()),
        ('scaler', MinMax()),
        ('one_hot_encoder', OneHotEncoding()),
        ('ordinal_encoder', OrdinalFeature()),
        ('smote', OverSample())
    ])

    df_pipeline = pipeline.fit_transform(df)
    return df_pipeline

In [16]:
df_train_clean = pipeline_ml(df_train)

In [17]:
df_test_clean = pipeline_ml(df_test)

In [18]:
print(df_test.shape)
print(df_test_clean.shape)

(7103, 16)
(13920, 50)


# Separando X e Y de treino

In [19]:
index_y = df_train_clean.columns.to_list().index('Risco_de_credito')
index_y

49

In [20]:
x_train, y_train = df_train_clean.drop(columns=['Risco_de_credito']), df_train_clean.iloc[:, index_y]
x_test, y_test = df_train_clean.drop(columns=['Risco_de_credito']), df_train_clean.iloc[:, index_y]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(41760, 49) (41760,)
(41760, 49) (41760,)


In [21]:
x_test.columns

Index(['Tem_carro', 'Tem_casa_propria', 'Tem_telefone_trabalho',
       'Tem_telefone_fixo', 'Tem_email', 'Idade', 'Anos_empregado',
       'Tamanho_familia', 'Rendimento_anual', 'Grau_escolaridade',
       'Categoria_de_renda_Associado comercial',
       'Categoria_de_renda_Empregado', 'Categoria_de_renda_Estudante',
       'Categoria_de_renda_Pensionista', 'Categoria_de_renda_Servidor público',
       'Estado_civil_Casado', 'Estado_civil_Divorciado',
       'Estado_civil_Solteiro', 'Estado_civil_União-estável',
       'Estado_civil_Viúvo', 'Moradia_Apartamento alugado',
       'Moradia_Apartamento comercial', 'Moradia_Casa/apartamento próprio',
       'Moradia_Cooperativa habitacional', 'Moradia_Habitação pública ',
       'Moradia_Mora com os pais', 'Ocupacao_Alta tecnologia',
       'Ocupacao_Associado comercial', 'Ocupacao_Baixa qualificação',
       'Ocupacao_Construção Civil', 'Ocupacao_Contabilidade',
       'Ocupacao_Corretor imobiliário', 'Ocupacao_Cozinha',
       'Ocupacao_

In [22]:
df_train_clean.to_csv('../../dados/credit_score/df_train_clean.csv', index=False)
df_test_clean.to_csv('../../dados/credit_score/df_test_clean.csv', index=False)